In [ ]:
import numpy as np

import matplotlib.pyplot as pl
import matplotlib as mpl

pl.style.use('fivethirtyeight')


#mpl.rc('figure', figsize=(8,6))

#mpl.rc('figure', figsize=(14,12))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'


font = {'size'   : 13}

mpl.rc('font', **font)

mpl.rcParams['lines.linewidth']=3
mpl.rcParams['axes.labelsize']=16


import os
import re

from scipy.io import loadmat
from scipy.ndimage import gaussian_filter1d
from scipy.signal import welch

import scipy.signal as sg

from functools import partial


In [ ]:
data_folder='../../Data/AS-2021_12_13-ChinQ336_fmaskedCAP_normal'

listFiles = os.listdir(data_folder)

listFilesMat=[]

for filename in listFiles:
    if re.match('.*.mat', filename):
        listFilesMat.append(filename)
listFilesMat.sort()

### Functions

In [ ]:
#filter ind if necessary
begin_ind=7 
end_ind=1486 #10 runs

pic_numbers_ignore=[71]

In [ ]:
def split_name(filename):
    '''returns (picNumber, masker filename)'''
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', filename)
    assert m, f'{filename} did not match'
    p, name= m.groups()
    return int(p), name

map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', fln)
    if m:
        p, name = split_name(fln)

        if name not in map_table:
            map_table[name]=[p]
        else:
            li=map_table[name]
            li.append(p)
            map_table[name]=li
        
filtered_map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    
    
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', fln)
    if m:
        p, name = split_name(fln)
        if p>=begin_ind and p<=end_ind and p not in pic_numbers_ignore:

            if name not in filtered_map_table:
                filtered_map_table[name]=[p]
            else:
                li=filtered_map_table[name]
                li.append(p)
                filtered_map_table[name]=li
    
    

In [ ]:
def loadPic(n):
    found=False
    for filename in listFilesMat:
        if f'p{str(n).zfill(4)}' in filename:
            found=True
            break
    assert found, f'pic {n} not found.'
    arr= loadmat(f'{data_folder}/{filename}')
    return arr['data_struct']

def get_info_pic(pic):
    #avg
    arr=pic['AD_Data'][0][0]['AD_Avg_V'][0][0][0]
    
    #HACK only C or R
    '''
    all_data=pic['AD_Data'][0][0]['AD_All_V'][0][0]
    arr=np.zeros_like(all_data[0])
    for i in range(len(all_data)):
        if i%2==1:
            arr+=all_data[i]
    arr/=len(all_data)/2
    '''

    return {'arr':arr,
            'XstartPlot_ms':pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XstartPlot_ms'][0][0][0],
            'XendPlot_ms': pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XendPlot_ms'][0][0][0],
           'name':pic['Stimuli'][0][0]['masker'][0][0]['name'][0][0]}


    


In [ ]:

def plotPic(picDic=None, n=None, label='', tmin=0, baselineSig=None):
    '''
    Plot pic given picture dictionary or pic number. 
    Args:
        baseline: Picture dictionary for baseline signal'''
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    pic_info=get_info_pic(picDic)
    if label=='':
        label=pic_info['name']
        
    
    val=pic_info['arr']
    
    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    if baselineSig is not None:
        val=val-baselineSig
    #val=np.cumsum(val) #HACK!!
    pl.plot(t, val, label=label)
    pl.xlim([tmin,  pic_info['XendPlot_ms']])
    
def getMaskerSig(dic, name):
    
    picNums=dic[name]
    firstPic=True
    for picNum in picNums:
        picDic=loadPic(picNum)
        
    
        pic_info=get_info_pic(picDic)
    
        if firstPic:
            #val=np.sum(picDic['valAll'][1::2], axis=0)
            val=pic_info['arr']
            
            firstPic=False
        else:
            #val+=np.sum(picDic['valAll'][1::2], axis=0)
            val+=pic_info['arr']
    val/=len(picNums)
    return val
    
def plotMasker(dic, name, tmin=5, tmax=11, baseline=None, corr_drift=False):
    '''
    Plot avg signal for masker given masker name and dictionary of picNums 
    Args:
        baseline: signal for baseline signal
        corr_drift: not set for this expe'''
    label=name
    val=getMaskerSig(dic, name)
    
    picNums=dic[name]
    for picNum in picNums:
        picDic=loadPic(picNum)
        pic_info=get_info_pic(picDic)
        
        break

    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    if baseline is not None:
        val=val-baseline
        
    #val=gaussian_filter1d(val, 0.02/(t[1]-t[0]))  #HACK filter
    #HACK
    #val=np.cumsum(val) #HACK!!
    #val-=val[300] #HACK!!
    pl.plot(t, val*1e3, label=label)
    if tmax==0:
        tmax=pic_info['XendPlot_ms']
    pl.xlim([tmin, tmax])
    pl.xlabel('t (ms)')
    if baseline is None:
        pl.ylabel('Amplitude (µV)')
    else:
        
        pl.ylabel('Amplitude difference (µV)')
    
'''
def getMaskerAtten(picDic=None, n=None):
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    ans=picDic['masker_atten_dB']
    return ans[0][0]
'''

\[new\]: find click artifacts 

In [ ]:

def findPicsArtifacts(begin_ind, end_ind, t_min=4e-3, t_max=5.2e-3, amp_min=80e-6):
    
    res=[]
    for n in range(begin_ind, end_ind+1):
        picDic=loadPic(n)
        pic_info=get_info_pic(picDic)

        val=pic_info['arr']
        t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))

        ind_t_min=np.sum(t<t_min*1e3)
        ind_t_max=np.sum(t<t_max*1e3)
        if np.any(np.abs(val[ind_t_min:ind_t_max])*1e-3>amp_min):
            res.append(n)

    return res

In [ ]:
#findPicsArtifacts(begin_ind, end_ind)  #t_min=4e-3, t_max=5.2e-3, amp_min=80e-6

#returns 71

In [ ]:
plotPic(n=71)

### Show examples of signals

In [ ]:
dic=filtered_map_table

In [ ]:
names=list(dic.keys())
names.sort()

#print(names)

In [ ]:
broadband=getMaskerSig(dic, 'broadband_noise')
nomasker=getMaskerSig(dic, 'nomasker_bis')

In [ ]:
broadband_sig=getMaskerSig(dic, 'broadband_noise')+getMaskerSig(dic, 'broadband_noise_bis')+getMaskerSig(dic, 'broadband_noise_bis_bis')
broadband_sig/=3

nomasker_sig=getMaskerSig(dic, 'nomasker')+getMaskerSig(dic, 'nomasker_bis')
nomasker_sig/=2

In [ ]:
info=get_info_pic(loadPic(100))
XstartPlot_ms=info['XstartPlot_ms']

XendPlot_ms=info['XendPlot_ms']
t=np.linspace(XstartPlot_ms, XendPlot_ms, num=len(broadband_sig))
plotMasker(dic, 'broadband_noise')


plotMasker(dic, 'broadband_noise_bis')

plotMasker(dic, 'broadband_noise_bis_bis')

pl.plot(t, broadband_sig*1e3, label='broadband average')


plotMasker(dic, 'nomasker')

plotMasker(dic, 'nomasker_bis')

pl.plot(t, nomasker_sig*1e3, label='nomasker average')


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

analysis noise (useless if no periodic background noise)

### Narrow-band analysis

In [ ]:
#plotMasker(dic, 'nomasker')
plotMasker(dic, '1_hp_10000Hz')
plotMasker(dic, '2_hp_9000Hz')
plotMasker(dic, '3_hp_8000Hz')
plotMasker(dic, '4_hp_7000Hz')
plotMasker(dic, '5_hp_6000Hz')
plotMasker(dic, '6_hp_5000Hz')
plotMasker(dic, '7_hp_4000Hz')
plotMasker(dic, '8_hp_3200Hz')
plotMasker(dic, '9_hp_2400Hz')
plotMasker(dic, '10_hp_1800Hz')
plotMasker(dic, '11_hp_1500Hz')
plotMasker(dic, '12_hp_1200Hz')
#plotMasker(dic, 'broadband_noise')  #5 dB higher
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plotMasker(dic, 'nomasker')
plotMasker(dic, '1_hp_10000Hz', baseline=broadband_sig)
plotMasker(dic, '2_hp_9000Hz', baseline=broadband_sig)
plotMasker(dic, '3_hp_8000Hz', baseline=broadband_sig)
plotMasker(dic, '4_hp_7000Hz', baseline=broadband_sig)
plotMasker(dic, '5_hp_6000Hz', baseline=broadband_sig)
plotMasker(dic, '6_hp_5000Hz', baseline=broadband_sig)
plotMasker(dic, '7_hp_4000Hz', baseline=broadband_sig)
plotMasker(dic, '8_hp_3200Hz', baseline=broadband_sig)
plotMasker(dic, '9_hp_2400Hz', baseline=broadband_sig)
plotMasker(dic, '10_hp_1800Hz', baseline=broadband_sig)
plotMasker(dic, '11_hp_1500Hz', baseline=broadband_sig)
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')  #5 dB higher
#pl.xlim([4.5,10.4])

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#pl.savefig('narrow-band.svg')

In [ ]:
plotMasker(dic, '1_hp_10000Hz', baseline=getMaskerSig(dic, '2_hp_9000Hz'))
plotMasker(dic, '2_hp_9000Hz', baseline=getMaskerSig(dic, '3_hp_8000Hz'))
plotMasker(dic, '3_hp_8000Hz', baseline=getMaskerSig(dic, '4_hp_7000Hz'))
plotMasker(dic, '4_hp_7000Hz', baseline=getMaskerSig(dic, '5_hp_6000Hz'))
plotMasker(dic, '5_hp_6000Hz', baseline=getMaskerSig(dic, '6_hp_5000Hz'))
plotMasker(dic, '6_hp_5000Hz', baseline=getMaskerSig(dic, '7_hp_4000Hz'))

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plotMasker(dic, 'nomasker')
plotMasker(dic, '7_hp_4000Hz', baseline=getMaskerSig(dic, '8_hp_3200Hz'))
plotMasker(dic, '8_hp_3200Hz', baseline=getMaskerSig(dic, '9_hp_2400Hz'))
plotMasker(dic, '9_hp_2400Hz', baseline=getMaskerSig(dic, '10_hp_1800Hz'))
plotMasker(dic, '10_hp_1800Hz', baseline=getMaskerSig(dic, '11_hp_1500Hz'))
plotMasker(dic, '11_hp_1500Hz', baseline=getMaskerSig(dic, '12_hp_1200Hz'))
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)

#plotMasker(dic, 'broadband_noise')  
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plotMasker(dic, 'nomasker')
#plotMasker(dic, '1_hp_10000Hz', baseline=getMaskerSig(dic, '3_hp_8000Hz'))
plotMasker(dic, '2_hp_9000Hz', baseline=getMaskerSig(dic, '4_hp_7000Hz'))
#plotMasker(dic, '3_hp_8000Hz', baseline=getMaskerSig(dic, '5_hp_6000Hz'))
plotMasker(dic, '4_hp_7000Hz', baseline=getMaskerSig(dic, '6_hp_5000Hz'))
#plotMasker(dic, '5_hp_6000Hz', baseline=getMaskerSig(dic, '7_hp_4000Hz'))
plotMasker(dic, '6_hp_5000Hz', baseline=getMaskerSig(dic, '8_hp_3200Hz'))
#plotMasker(dic, '7_hp_4000Hz', baseline=getMaskerSig(dic, '9_hp_2400Hz'))
plotMasker(dic, '8_hp_3200Hz', baseline=getMaskerSig(dic, '10_hp_1800Hz'))
#plotMasker(dic, '9_hp_2400Hz', baseline=getMaskerSig(dic, '11_hp_1500Hz'))
plotMasker(dic, '10_hp_1800Hz', baseline=getMaskerSig(dic, '12_hp_1200Hz'))
#plotMasker(dic, '11_hp_1500Hz', baseline=baseline=broadband_sig)
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')  #5 dB higher
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Notch 8k

In [ ]:

#plotMasker(dic, '2_notch8000_bw2300_55dB')
#plotMasker(dic, '3_notch8000_bw2300_50dB')  #removed
#plotMasker(dic, '4_notch8000_bw2300_45dB')
plotMasker(dic, '5_notch8000_bw2300_40dB')
plotMasker(dic, '6_notch8000_bw2300_35dB')
plotMasker(dic, '7_notch8000_bw2300_32dB')
plotMasker(dic, '7_notch8000_bw2300_29dB')
plotMasker(dic, '8_notch8000_bw2300_26dB')
plotMasker(dic, '9_notch8000_bw2300_23dB')


plotMasker(dic, '10_notch8000_bw2300_17dB') #the new one
plotMasker(dic, 'broadband_noise')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:

#plotMasker(dic, '2_notch8000_bw2300_55dB', baseline=broadband_sig)
#plotMasker(dic, '3_notch8000_bw2300_50dB', baseline=broadband_sig)
#plotMasker(dic, '4_notch8000_bw2300_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch8000_bw2300_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch8000_bw2300_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch8000_bw2300_32dB', baseline=broadband_sig)
plotMasker(dic, '7_notch8000_bw2300_29dB', baseline=broadband_sig)
plotMasker(dic, '8_notch8000_bw2300_26dB', baseline=broadband_sig)
plotMasker(dic, '9_notch8000_bw2300_23dB', baseline=broadband_sig)

plotMasker(dic, '10_notch8000_bw2300_17dB', baseline=broadband_sig)

#plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch8000_bw2300_55dB',
# '3_notch8000_bw2300_50dB',
# '4_notch8000_bw2300_45dB',
 '5_notch8000_bw2300_40dB',
 '6_notch8000_bw2300_35dB',
 '7_notch8000_bw2300_32dB',
 '7_notch8000_bw2300_29dB',
 '8_notch8000_bw2300_26dB',
 '9_notch8000_bw2300_23dB',
            '10_notch8000_bw2300_17dB']
           
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
    
    
#HACK
cap[-1]=-cap[-1]

pl.plot(-np.array([40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch7600_bw1100.json',
'2-notch7800_bw1300.json',
'3-notch8000_bw1400.json',
'4-notch8200_bw1300.json',
'5-notch8200_bw1500.json',
'6-notch7900_bw1600.json',
'7-notch8100_bw1200.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st)
    
pl.legend()

In [ ]:
fln_list=['1-notch7600_bw1100.json',
'2-notch7800_bw1300.json',
'3-notch8000_bw1400.json',
'4-notch8200_bw1300.json',
'5-notch8200_bw1500.json',
'6-notch7900_bw1600.json',
'7-notch8100_bw1200.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

Maskers added

In [ ]:
fln_list=['2-notch7300_1600_30dB.json', '3-notch7300_1600_25dB.json', '4-notch8300_2000_26dB.json',
 '5-notch8300_2000_23dB.json']

fln_list+=['1-notch7400_bw1000.json',
'2-notch8400_bw1200.json',
'3-notch7800_bw1800.json',
'8-notch8100_bw2000.json']

masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Notch 6k

In [ ]:
#plotMasker(dic, '2_notch6000_bw2000_55dB')
#plotMasker(dic, '3_notch6000_bw2000_50dB')
plotMasker(dic, '4_notch6000_bw2000_45dB')
plotMasker(dic, '5_notch6000_bw2000_40dB')
plotMasker(dic, '6_notch6000_bw2000_35dB')
plotMasker(dic, '7_notch6000_bw2000_32dB')
plotMasker(dic, '8_notch6000_bw2000_29dB')
plotMasker(dic, '9_notch6000_bw2000_26dB')
plotMasker(dic, '10_notch6000_bw2000_23dB')

plotMasker(dic, '11_notch6000_bw2000_17dB')

plotMasker(dic, 'broadband_noise')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plotMasker(dic, '2_notch6000_bw2000_55dB', baseline=broadband_sig)
#plotMasker(dic, '3_notch6000_bw2000_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch6000_bw2000_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch6000_bw2000_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch6000_bw2000_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch6000_bw2000_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch6000_bw2000_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch6000_bw2000_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch6000_bw2000_23dB', baseline=broadband_sig)

plotMasker(dic, '11_notch6000_bw2000_17dB', baseline=broadband_sig)  #re-added

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch6000_bw2000_55dB',
#'3_notch6000_bw2000_50dB',
'4_notch6000_bw2000_45dB',
'5_notch6000_bw2000_40dB',
'6_notch6000_bw2000_35dB',
'7_notch6000_bw2000_32dB',
'8_notch6000_bw2000_29dB',
'9_notch6000_bw2000_26dB',
'10_notch6000_bw2000_23dB', 
'11_notch6000_bw2000_17dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([45,40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch5900_bw900.json',
'2-notch6200_bw1000.json',
'3-notch6000_bw1100.json',
'4-notch6100_bw1200.json',
'7-notch5900_bw1300.json',
'8-notch5800_bw1400.json',
'9-notch6300_bw1100.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
fln_list=['1-notch5900_bw900.json',
'2-notch6200_bw1000.json',
'3-notch6000_bw1100.json',
'4-notch6100_bw1200.json',
'7-notch5900_bw1300.json',
'8-notch5800_bw1400.json',
'9-notch6300_bw1100.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


New maskers

In [ ]:
fln_list=['1-notch5800_1400_30dB.json',
'2-notch5800_1400_25dB.json',
'3-notch6500_1400_26dB.json',
'4-notch6500_1400_23dB.json',
'5-notch7000_1600_26dB.json',
'6-notch7000_1600_23dB.json',
'1-notch6600_bw900.json',
'2-notch7000_bw1200.json']


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


### Notch 5k

In [ ]:
#plotMasker(dic, '2_notch5000_bw2000_55dB')
#plotMasker(dic, '3_notch5000_bw2000_50dB')
plotMasker(dic, '4_notch5000_bw2000_45dB')
plotMasker(dic, '5_notch5000_bw2000_40dB')
plotMasker(dic, '6_notch5000_bw2000_35dB')
plotMasker(dic, '7_notch5000_bw2000_32dB')
plotMasker(dic, '8_notch5000_bw2000_29dB')
plotMasker(dic, '9_notch5000_bw2000_26dB')
plotMasker(dic, '10_notch5000_bw2000_23dB')
plotMasker(dic, '11_notch5000_bw2000_17dB')
plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
#plotMasker(dic, '2_notch5000_bw2000_55dB', baseline=broadband_sig)
#plotMasker(dic, '3_notch5000_bw2000_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch5000_bw2000_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch5000_bw2000_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch5000_bw2000_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch5000_bw2000_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch5000_bw2000_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch5000_bw2000_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch5000_bw2000_23dB', baseline=broadband_sig)

plotMasker(dic, '11_notch5000_bw2000_17dB', baseline=broadband_sig)
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#pl.savefig('notch5000.svg')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch5000_bw2000_55dB',
'4_notch5000_bw2000_45dB',
'5_notch5000_bw2000_40dB',
'6_notch5000_bw2000_35dB',
'7_notch5000_bw2000_32dB',
'8_notch5000_bw2000_29dB',
'9_notch5000_bw2000_26dB',
'10_notch5000_bw2000_23dB',
'11_notch5000_bw2000_17dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
    
#HACK
cap[-1]=-cap[-1]


pl.plot(-np.array([ 45,40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch4800_bw900.json',
#'2-notch5000_bw1000.json', #removed
'3-notch5100_bw1200.json',
'4-notch4800_bw1400.json',
'5-notch5100_bw1100.json',
'6-notch4900_bw1300.json',
'7-notch5200_bw900.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
fln_list=['1-notch4800_bw900.json',
#'2-notch5000_bw1000.json',
'3-notch5100_bw1200.json',
'4-notch4800_bw1400.json',
'5-notch5100_bw1100.json',
'6-notch4900_bw1300.json',
'7-notch5200_bw900.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

New maskers

In [ ]:
fln_list=['1-notch4800_1200_30dB.json',
'2-notch4800_1200_25dB.json',
'3-notch5300_1200_26dB.json',
'2-notch5400_bw1000.json',
'3-notch5000_bw800.json']


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Notch 4k

In [ ]:
#plotMasker(dic, '2_notch4000_bw1700_55dB')
#plotMasker(dic, '3_notch4000_bw1700_50dB')
plotMasker(dic, '4_notch4000_bw1700_45dB')
plotMasker(dic, '5_notch4000_bw1700_40dB')
plotMasker(dic, '6_notch4000_bw1700_35dB')
plotMasker(dic, '7_notch4000_bw1700_32dB')
plotMasker(dic, '8_notch4000_bw1700_29dB')
plotMasker(dic, '9_notch4000_bw1700_26dB')
plotMasker(dic, '10_notch4000_bw1700_23dB')

plotMasker(dic, '11_notch400_bw1700_17dB')  #XXX be careful, typo in the name
plotMasker(dic, 'broadband_noise')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')



In [ ]:
#plotMasker(dic, '2_notch4000_bw1700_55dB', baseline=broadband_sig)
#plotMasker(dic, '3_notch4000_bw1700_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch4000_bw1700_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch4000_bw1700_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch4000_bw1700_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch4000_bw1700_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch4000_bw1700_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch4000_bw1700_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch4000_bw1700_23dB', baseline=broadband_sig)

plotMasker(dic, '11_notch400_bw1700_17dB', baseline=broadband_sig)

#plotMasker(dic, 'broadband_noise', baseline=broadband_sig)

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch4000_bw1700_55dB',
'4_notch4000_bw1700_45dB',
'5_notch4000_bw1700_40dB',
'6_notch4000_bw1700_35dB',
'7_notch4000_bw1700_32dB',
'8_notch4000_bw1700_29dB',
'9_notch4000_bw1700_26dB',
'10_notch4000_bw1700_23dB',
'11_notch400_bw1700_17dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
    
#HACK
cap[-1]=-cap[-1]


pl.plot(-np.array([ 45,40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch3800_bw800.json',
'2-notch4000_bw900.json',
'3-notch4000_bw1000.json',
'4-notch4200_bw1100.json',
'5-notch4200_bw800.json',
'6-notch3800_bw1200.json']

masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


New maskers

In [ ]:
fln_list=['1-notch3800_1100_30dB.json',
'2-notch3800_1100_25dB.json',
'3-notch4300_1100_26dB.json',
'4-notch4300_1100_23dB.json',
'1-notch4500_bw700.json']


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


### Notch 3k

In [ ]:
#plotMasker(dic, '2_notch3000_bw1500_55dB')
#plotMasker(dic, '4_notch3000_bw1500_50dB')
plotMasker(dic, '5_notch3000_bw1500_45dB')
plotMasker(dic, '6_notch3000_bw1500_40dB')
plotMasker(dic, '7_notch3000_bw1500_35dB')
plotMasker(dic, '8_notch3000_bw1500_32dB')
plotMasker(dic, '9_notch3000_bw1500_29dB')
plotMasker(dic, '10_notch3000_bw1500_26dB')
plotMasker(dic, '11_notch3000_bw1500_23dB')

plotMasker(dic, '12_notch300_bw1500_17dB')  #typo
plotMasker(dic, 'broadband_noise')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
#plotMasker(dic, '2_notch3000_bw1500_55dB', baseline=broadband_sig)
#plotMasker(dic, '4_notch3000_bw1500_50dB', baseline=broadband_sig)
plotMasker(dic, '5_notch3000_bw1500_45dB', baseline=broadband_sig)
plotMasker(dic, '6_notch3000_bw1500_40dB', baseline=broadband_sig)
plotMasker(dic, '7_notch3000_bw1500_35dB', baseline=broadband_sig)
plotMasker(dic, '8_notch3000_bw1500_32dB', baseline=broadband_sig)
plotMasker(dic, '9_notch3000_bw1500_29dB', baseline=broadband_sig)
plotMasker(dic, '10_notch3000_bw1500_26dB', baseline=broadband_sig)
plotMasker(dic, '11_notch3000_bw1500_23dB', baseline=broadband_sig)

plotMasker(dic, '12_notch300_bw1500_17dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch3000_bw1500_55dB'
#,'4_notch3000_bw1500_50dB'
'5_notch3000_bw1500_45dB'
,'6_notch3000_bw1500_40dB'
,'7_notch3000_bw1500_35dB'
,'8_notch3000_bw1500_32dB'
,'9_notch3000_bw1500_29dB'
,'10_notch3000_bw1500_26dB'
,'11_notch3000_bw1500_23dB',
 '12_notch300_bw1500_17dB']
    
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
    
#HACK
cap[-1]=-cap[-1]

pl.plot(-np.array([45,40,35,32,29,26,23, 17]), cap, label='max-min')

    

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch2900_bw600.json',
'2-notch3000_bw700.json',
'3-notch3300_bw700.json',
'4-notch3100_bw800.json',
'6-notch3000_bw900.json',
'7-notch3100_bw1000.json']


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


New maskers

In [ ]:
fln_list=['1-notch2600_900_30dB.json',
'2-notch2600_900_25dB.json',
'3-notch3400_900_26dB.json',
'4-notch3400_900_23dB.json',
'1-notch2800_bw800.json',
'2-notch3400_bw900.json']


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


### notch 2.2k

In [ ]:
#plotMasker(dic, '2_notch2200_bw1500_55dB')
#plotMasker(dic, '3_notch2200_bw1500_50dB')
plotMasker(dic, '4_notch2200_bw1500_45dB')
plotMasker(dic, '5_notch2200_bw1500_40dB')
plotMasker(dic, '6_notch2200_bw1500_35dB')
plotMasker(dic, '7_notch2200_bw1500_32dB')
plotMasker(dic, '8_notch2200_bw1500_29dB')
plotMasker(dic, '9_notch2200_bw1500_26dB')
plotMasker(dic, '10_notch2200_bw1500_23dB')

plotMasker(dic, '11_notch2200_bw1500_17dB')
plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


In [ ]:
#plotMasker(dic, '2_notch2200_bw1500_55dB', baseline=broadband_sig)
#plotMasker(dic, '3_notch2200_bw1500_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch2200_bw1500_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch2200_bw1500_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch2200_bw1500_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch2200_bw1500_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch2200_bw1500_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch2200_bw1500_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch2200_bw1500_23dB', baseline=broadband_sig)

plotMasker(dic, '11_notch2200_bw1500_17dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[#'2_notch2200_bw1500_55dB',
#'3_notch2200_bw1500_50dB',
'4_notch2200_bw1500_45dB',
'5_notch2200_bw1500_40dB',
'6_notch2200_bw1500_35dB',
'7_notch2200_bw1500_32dB',
'8_notch2200_bw1500_29dB',
'9_notch2200_bw1500_26dB',
'10_notch2200_bw1500_23dB',
            '11_notch2200_bw1500_17dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
    
    
#HACK
cap[-1]=-cap[-1]


pl.plot(-np.array([ 45,40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=['1-notch2000_bw600.json', #modified
'2-notch2200_bw800.json',
'3-notch2300_bw900.json',
          '3-notch2600_bw700.json', #added
'4-notch2500_bw600.json',  #modified
'5-notch2400_bw900.json']

masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Notch 1.5k

In [ ]:
#plotMasker(dic, '31_notch1500_bw1000_55dB')
#plotMasker(dic, '32_notch1500_bw1000_50dB')
plotMasker(dic, '33_notch1500_bw1000_45dB')
plotMasker(dic, '34_notch1500_bw1000_40dB')
plotMasker(dic, '35_notch1500_bw1000_35dB')
plotMasker(dic, '36_notch1500_bw1000_32dB')
plotMasker(dic, '37_notch1500_bw1000_29dB')
plotMasker(dic, '38_notch1500_bw1000_26dB')
plotMasker(dic, '39_notch1500_bw1000_23dB')
plotMasker(dic, '40_notch1500_bw1000_17dB')
plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plotMasker(dic, '31_notch1500_bw1000_55dB', baseline=broadband_sig)
#plotMasker(dic, '32_notch1500_bw1000_50dB', baseline=broadband_sig)
plotMasker(dic, '33_notch1500_bw1000_45dB', baseline=broadband_sig)
plotMasker(dic, '34_notch1500_bw1000_40dB', baseline=broadband_sig)
plotMasker(dic, '35_notch1500_bw1000_35dB', baseline=broadband_sig)
plotMasker(dic, '36_notch1500_bw1000_32dB', baseline=broadband_sig)
plotMasker(dic, '37_notch1500_bw1000_29dB', baseline=broadband_sig)
plotMasker(dic, '38_notch1500_bw1000_26dB', baseline=broadband_sig)
plotMasker(dic, '39_notch1500_bw1000_23dB', baseline=broadband_sig)
plotMasker(dic, '40_notch1500_bw1000_17dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]


masker_list=[#'31_notch1500_bw1000_55dB',
#'32_notch1500_bw1000_50dB',
'33_notch1500_bw1000_45dB',
'34_notch1500_bw1000_40dB',
'35_notch1500_bw1000_35dB',
'36_notch1500_bw1000_32dB',
'37_notch1500_bw1000_29dB',
'38_notch1500_bw1000_26dB',
'39_notch1500_bw1000_23dB',
            
'40_notch1500_bw1000_17dB']

for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
#HACK
cap[-1]=-cap[-1]
pl.plot(-np.array([45,40,35,32,29,26,23, 17]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
fln_list=[#'1-notch_1400_bw400.json',
'2-notch1500_bw500.json',
'3-notch1500_bw700.json',
'4-notch1600_bw600.json',
'5-notch1700_bw800.json',
#'6-notch1700_bw500.json',
'3-notch1300_bw400.json', #added
'4-notch1800_bw500.json'  #added
]


masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
#print(masker_list)

for masker_st in masker_list:
    plotMasker(dic, masker_st, baseline=broadband_sig)
    

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')